# Price Prediction

# Mounting Drive

In [0]:
!kill -9 -1

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pwd
!ls

/content
drive  sample_data


In [0]:
import os
PATH = os.getcwd()
print(PATH)

/content


In [0]:
data_path = PATH + '/drive/My Drive/JOB/'
data_path

'/content/drive/My Drive/JOB/'

# 1 Data Preprocessing

## 1.1 Reading the data

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
project_data = pd.read_csv(data_path + "dataset.csv")
project_data.head()

,Unique Id,Image,Brand,Title,Specifications,Highlights,Price,Rating
0,0697f1714bf848c4b1cacc39a46a4e8b,https://images-na.ssl-images-amazon.com/images...,RoadPro,RoadPro RP-100C 10' AM/FM Antenna Coaxial Cable,"[{'Brand': 'RoadPro'}, {'Model': 'RP-100C'}, {...",Make sure this fits by entering your model num...,12.99,4.7
1,f04481fec10c4a538634d01e04bb45ed,https://images-na.ssl-images-amazon.com/images...,YSense,5-6 Packs Baby Girl Cute Headband - Turban Hea...,[{}],"Material: Made of elastic synthetic cotton, so...",9.99,5.0
2,754577bc25b24641ab1b0aef5e19db73,https://images-na.ssl-images-amazon.com/images...,Wooden Camera,Wooden Camera - PL Mount (Weapon/Scarlet-W/Epi...,[{'Product Dimensions': '3.2 x 1.8 x 3.2 inche...,Make sure this fits by entering your model num...,379.18,3.3
3,b4c5f6d192124d70919946ea754cce56,https://images-na.ssl-images-amazon.com/images...,Global-store,Globalstore 12x25 Folding High-Grade Binocular...,[{'Product Dimensions': '3.9 x 3.9 x 2 inches'...,Make sure this fits by entering your model num...,19.97,5.0
4,b1a1b9c3926b4da1ab0040ea81b41e3e,https://images-na.ssl-images-amazon.com/images...,GBTIGER,"120 inch Portable Projection Screen with Bag, ...","[{'Brand Name': 'GBTIGER'}, {'Item Weight': '2...",Make sure this fits by entering your model num...,27.99,3.0


In [0]:
project_data.shape

(51500, 8)

In [0]:
project_data[project_data['Price'] == 'not_available'].shape[0]

3499

This means that for 3499 rows the price data is not available and that is what we have to predict.
This will comprise out test data

## 1.2 Remving data which is not useful PASS 1

### Title

In [0]:
project_data['Title'].isnull().value_counts()

False    51499
True         1
Name: Title, dtype: int64

Title is an important part of predicting the price as it gives a lot of information about the product so we will eliminate the row with no title

In [0]:
project_data = project_data[project_data['Title'].isnull() == False]

**Eliminating the rows with less data(no.of words) in title**

In [0]:
project_data_sorted = project_data[project_data['Title'].apply(lambda x: len(x.split())>=4)]
print("After removal of products with short Title:", project_data_sorted.shape[0])

After removal of products with short Title: 50494


### Brand

In [0]:
project_data_sorted['Brand'].isnull().value_counts()

False    50491
True         3
Name: Brand, dtype: int64

In [0]:
project_data_sorted = project_data_sorted[~project_data_sorted['Brand'].isnull()]
print("After removal of products with no brand name:", project_data_sorted.shape[0])

After removal of products with no brand name: 50491


### Highlights

In [0]:
project_data_sorted['Highlights'].isnull().value_counts()

False    50489
True         2
Name: Highlights, dtype: int64

In [0]:
project_data_sorted = project_data_sorted[~project_data_sorted['Highlights'].isnull()]
print("After removal of products with no Highlights:", project_data_sorted.shape[0])

After removal of products with no Highlights: 50489


### Image

In [0]:
project_data_sorted['Image'].isnull().value_counts()

False    47154
True      3335
Name: Image, dtype: int64

In [0]:
project_data_sorted = project_data_sorted[~project_data_sorted['Image'].isnull()]
print("After removal of products with no Image:", project_data_sorted.shape[0])

After removal of products with no Image: 47154


In [0]:
project_data_sorted[project_data_sorted['Image'].apply(lambda x:str(x).startswith("https"))].shape    #check if any of them was not_available

(47154, 8)

### Ratings

In [0]:
project_data_sorted['Rating'].isnull().value_counts()

False    47154
Name: Rating, dtype: int64

### Specifications

In [0]:
project_data_sorted['Specifications'].isnull().value_counts()

False    47154
Name: Specifications, dtype: int64

In [0]:
project_data_sorted[project_data_sorted['Specifications'] == '[{}]'].shape

(17954, 8)

This means that not all the data has specifications

### Unique ID

In [0]:
project_data_sorted['Unique Id'].isnull().value_counts()

False    47154
Name: Unique Id, dtype: int64

**At the end of first parsing to eliminate the unuseful data we are left with 47154 rows of data**

## 1.3 Remving data which is not useful PASS 2

### Brand

In [0]:
project_data_sorted[project_data_sorted['Brand'] == 'not_available'].shape

(72, 8)

In [0]:
project_data_sorted.loc[project_data_sorted['Brand'] == 'not_available',['Brand']] = ''

In [0]:
project_data_sorted[project_data_sorted['Brand'] == 'not_available'].shape

(0, 8)

### Highlights

In [0]:
project_data_sorted[project_data_sorted['Highlights'].apply(lambda x:x.startswith('Make sure this fits by entering your model number.\n'))].shape

(20067, 8)

In [0]:
from tqdm import tqdm
updated_highlights = []
for i in tqdm(project_data_sorted['Highlights']):
    if i == 'not_available':
        i = ''
    elif i.startswith('Make sure this fits by entering your model number.\n'):
        index = i.find('\n')
        i = i[index+2:]
    updated_highlights.append(i)
len(updated_highlights)

100%|██████████| 47154/47154 [00:00<00:00, 730000.15it/s]


47154

In [0]:
project_data_sorted['Clean_highlights'] = updated_highlights
project_data_sorted.drop(['Highlights'], axis=1, inplace=True)
project_data_sorted.head()

,Unique Id,Image,Brand,Title,Specifications,Price,Rating,Clean_highlights
0,0697f1714bf848c4b1cacc39a46a4e8b,https://images-na.ssl-images-amazon.com/images...,RoadPro,RoadPro RP-100C 10' AM/FM Antenna Coaxial Cable,"[{'Brand': 'RoadPro'}, {'Model': 'RP-100C'}, {...",12.99,4.7,G-58A/U Coax Cable with PL-259 to Motorola Plu...
1,f04481fec10c4a538634d01e04bb45ed,https://images-na.ssl-images-amazon.com/images...,YSense,5-6 Packs Baby Girl Cute Headband - Turban Hea...,[{}],9.99,5.0,"Material: Made of elastic synthetic cotton, so..."
2,754577bc25b24641ab1b0aef5e19db73,https://images-na.ssl-images-amazon.com/images...,Wooden Camera,Wooden Camera - PL Mount (Weapon/Scarlet-W/Epi...,[{'Product Dimensions': '3.2 x 1.8 x 3.2 inche...,379.18,3.3,L Mount for RED Epic and Scarlet Cameras.
3,b4c5f6d192124d70919946ea754cce56,https://images-na.ssl-images-amazon.com/images...,Global-store,Globalstore 12x25 Folding High-Grade Binocular...,[{'Product Dimensions': '3.9 x 3.9 x 2 inches'...,19.97,5.0,ompact binoculars for bird watching: the mini ...
4,b1a1b9c3926b4da1ab0040ea81b41e3e,https://images-na.ssl-images-amazon.com/images...,GBTIGER,"120 inch Portable Projection Screen with Bag, ...","[{'Brand Name': 'GBTIGER'}, {'Item Weight': '2...",27.99,3.0,120 inch Projector Screen】: 16:9 screen format...


### Specifications

In [0]:
project_data_sorted['Specifications'][0]

"[{'Brand': 'RoadPro'}, {'Model': 'RP-100C'}, {'Item Weight': '3.2 ounces'}, {'Product Dimensions': '5 x 5 x 5 inches'}, {'Item model number': 'RP-100C'}, {'Manufacturer Part Number': 'RP-100C'}, {'Folding': 'No'}]"

In [0]:
import re
updated_specifications = []
pattern = '[^A-Za-z0-9,-]'
replace = ' '
for i in tqdm(project_data_sorted['Specifications']):
    result = re.sub(pattern, replace, i)
    result = result.strip().replace("  "," ")
    updated_specifications.append(result)
len(updated_specifications)

100%|██████████| 47154/47154 [00:01<00:00, 42056.00it/s]


47154

In [0]:
project_data_sorted['Clean_specifications'] = updated_specifications
project_data_sorted.drop(['Specifications'], axis=1, inplace=True)
project_data_sorted.head()

,Unique Id,Image,Brand,Title,Price,Rating,Clean_highlights,Clean_specifications
0,0697f1714bf848c4b1cacc39a46a4e8b,https://images-na.ssl-images-amazon.com/images...,RoadPro,RoadPro RP-100C 10' AM/FM Antenna Coaxial Cable,12.99,4.7,G-58A/U Coax Cable with PL-259 to Motorola Plu...,"Brand RoadPro , Model RP-100C , Item Weigh..."
1,f04481fec10c4a538634d01e04bb45ed,https://images-na.ssl-images-amazon.com/images...,YSense,5-6 Packs Baby Girl Cute Headband - Turban Hea...,9.99,5.0,"Material: Made of elastic synthetic cotton, so...",
2,754577bc25b24641ab1b0aef5e19db73,https://images-na.ssl-images-amazon.com/images...,Wooden Camera,Wooden Camera - PL Mount (Weapon/Scarlet-W/Epi...,379.18,3.3,L Mount for RED Epic and Scarlet Cameras.,"Product Dimensions 3 2 x 1 8 x 3 2 inches , ..."
3,b4c5f6d192124d70919946ea754cce56,https://images-na.ssl-images-amazon.com/images...,Global-store,Globalstore 12x25 Folding High-Grade Binocular...,19.97,5.0,ompact binoculars for bird watching: the mini ...,"Product Dimensions 3 9 x 3 9 x 2 inches , It..."
4,b1a1b9c3926b4da1ab0040ea81b41e3e,https://images-na.ssl-images-amazon.com/images...,GBTIGER,"120 inch Portable Projection Screen with Bag, ...",27.99,3.0,120 inch Projector Screen】: 16:9 screen format...,"Brand Name GBTIGER , Item Weight 2 1 pounds..."


# 2 Preparing for modelling

Now the title of a product tells us much more about that product than anything else so we will increase the weight of the title and make a single column for all the text data

In [0]:
project_data_sorted['Text'] = 3*(project_data_sorted['Title']+' ') + 2*(project_data_sorted['Clean_specifications']+' ') + project_data_sorted['Clean_highlights']
project_data_sorted.head()

,Unique Id,Image,Brand,Title,Price,Rating,Clean_highlights,Clean_specifications,Text
0,0697f1714bf848c4b1cacc39a46a4e8b,https://images-na.ssl-images-amazon.com/images...,RoadPro,RoadPro RP-100C 10' AM/FM Antenna Coaxial Cable,12.99,4.7,G-58A/U Coax Cable with PL-259 to Motorola Plu...,"Brand RoadPro , Model RP-100C , Item Weigh...",RoadPro RP-100C 10' AM/FM Antenna Coaxial Cabl...
1,f04481fec10c4a538634d01e04bb45ed,https://images-na.ssl-images-amazon.com/images...,YSense,5-6 Packs Baby Girl Cute Headband - Turban Hea...,9.99,5.0,"Material: Made of elastic synthetic cotton, so...",,5-6 Packs Baby Girl Cute Headband - Turban Hea...
2,754577bc25b24641ab1b0aef5e19db73,https://images-na.ssl-images-amazon.com/images...,Wooden Camera,Wooden Camera - PL Mount (Weapon/Scarlet-W/Epi...,379.18,3.3,L Mount for RED Epic and Scarlet Cameras.,"Product Dimensions 3 2 x 1 8 x 3 2 inches , ...",Wooden Camera - PL Mount (Weapon/Scarlet-W/Epi...
3,b4c5f6d192124d70919946ea754cce56,https://images-na.ssl-images-amazon.com/images...,Global-store,Globalstore 12x25 Folding High-Grade Binocular...,19.97,5.0,ompact binoculars for bird watching: the mini ...,"Product Dimensions 3 9 x 3 9 x 2 inches , It...",Globalstore 12x25 Folding High-Grade Binocular...
4,b1a1b9c3926b4da1ab0040ea81b41e3e,https://images-na.ssl-images-amazon.com/images...,GBTIGER,"120 inch Portable Projection Screen with Bag, ...",27.99,3.0,120 inch Projector Screen】: 16:9 screen format...,"Brand Name GBTIGER , Item Weight 2 1 pounds...","120 inch Portable Projection Screen with Bag, ..."


In [0]:
print(project_data_sorted['Title'][0])
print(project_data_sorted['Clean_specifications'][0])
print(project_data_sorted['Clean_highlights'][0])
print(project_data_sorted['Text'][0])

RoadPro RP-100C 10' AM/FM Antenna Coaxial Cable
Brand  RoadPro ,  Model  RP-100C ,  Item Weight  3 2 ounces ,  Product Dimensions  5 x 5 x 5 inches ,  Item model number  RP-100C ,  Manufacturer Part Number  RP-100C ,  Folding  No
G-58A/U Coax Cable with PL-259 to Motorola Plug
10' AM/FM antenna coaxial cable
Black
RoadPro RP-100C 10' AM/FM Antenna Coaxial Cable RoadPro RP-100C 10' AM/FM Antenna Coaxial Cable RoadPro RP-100C 10' AM/FM Antenna Coaxial Cable Brand  RoadPro ,  Model  RP-100C ,  Item Weight  3 2 ounces ,  Product Dimensions  5 x 5 x 5 inches ,  Item model number  RP-100C ,  Manufacturer Part Number  RP-100C ,  Folding  No Brand  RoadPro ,  Model  RP-100C ,  Item Weight  3 2 ounces ,  Product Dimensions  5 x 5 x 5 inches ,  Item model number  RP-100C ,  Manufacturer Part Number  RP-100C ,  Folding  No G-58A/U Coax Cable with PL-259 to Motorola Plug
10' AM/FM antenna coaxial cable
Black


In [0]:
project_data_sorted.drop(['Clean_specifications','Clean_highlights','Title'], axis=1, inplace=True)
project_data_sorted.head()

,Unique Id,Image,Brand,Price,Rating,Text
0,0697f1714bf848c4b1cacc39a46a4e8b,https://images-na.ssl-images-amazon.com/images...,RoadPro,12.99,4.7,RoadPro RP-100C 10' AM/FM Antenna Coaxial Cabl...
1,f04481fec10c4a538634d01e04bb45ed,https://images-na.ssl-images-amazon.com/images...,YSense,9.99,5.0,5-6 Packs Baby Girl Cute Headband - Turban Hea...
2,754577bc25b24641ab1b0aef5e19db73,https://images-na.ssl-images-amazon.com/images...,Wooden Camera,379.18,3.3,Wooden Camera - PL Mount (Weapon/Scarlet-W/Epi...
3,b4c5f6d192124d70919946ea754cce56,https://images-na.ssl-images-amazon.com/images...,Global-store,19.97,5.0,Globalstore 12x25 Folding High-Grade Binocular...
4,b1a1b9c3926b4da1ab0040ea81b41e3e,https://images-na.ssl-images-amazon.com/images...,GBTIGER,27.99,3.0,"120 inch Portable Projection Screen with Bag, ..."


### Image

In [0]:
project_data_sorted.shape

(47154, 8)

In [0]:
import os
files = os.listdir('images final/')
u_id = []
for i in files:
    u_id.append(i.split('.')[0])
    
len(set(list(u_id)))

47154

In [0]:
from PIL import Image
import requests
from io import BytesIO

for index, row in project_data_sorted.iterrows():
    if(row['Unique Id'] not in u_id):
        url = row['Image']
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        img.save('images/'+row['Unique Id']+'.jpeg')

In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from sklearn.metrics import pairwise_distances
import matplotlib.pyplot as plt
import requests
from PIL import Image
import pandas as pd
import pickle

Using TensorFlow backend.


In [0]:
# dimensions of our images.
img_width, img_height = 224, 224

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'images final'
nb_train_samples = 47154
epochs = 50
batch_size = 1


def save_bottlebeck_features():
    
    #Function to compute VGG-16 CNN for image feature extraction.
    
    all_id = []
    datagen = ImageDataGenerator(rescale=1. / 255)
    
    # build the VGG16 network
    model = applications.vgg16.VGG16(include_top=False, weights='imagenet')
    generator = datagen.flow_from_directory(
        r'D:\JOB\images',
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    
    for i in generator.filenames:
        all_id.append(i.split('.')[0])

    bottleneck_features_train = model.predict_generator(generator, nb_train_samples // batch_size)
    print(bottleneck_features_train.shape)
    bottleneck_features_train = bottleneck_features_train.reshape((47154,25088))
    
    np.save(open('dataset_cnn_features.npy', 'wb'), bottleneck_features_train)
    np.save(open('dataset_cnn_feature_id.npy', 'wb'), np.array(all_id))
    

save_bottlebeck_features()

Found 47154 images belonging to 1 classes.
(47154, 7, 7, 512)


In [0]:
bottleneck_features_train = np.load('dataset_cnn_features.npy')
all_id = np.load('dataset_cnn_feature_id.npy')

In [0]:
col_names = []
for i in range(25088):
    col_names.append('feature '+str(i+1))
col_names[:5]

['feature 1', 'feature 2', 'feature 3', 'feature 4', 'feature 5']

In [0]:
bottleneck_dataframe = pd.DataFrame(bottleneck_features_train, index=None, columns=col_names)
bottleneck_dataframe.head()

,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,feature 9,feature 10,...,feature 25079,feature 25080,feature 25081,feature 25082,feature 25083,feature 25084,feature 25085,feature 25086,feature 25087,feature 25088
0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.059623,...,1.789614,0.749271,0.359782,0.0,0.0,0.452725,0.096178,0.0,0.750572,0.0
1,2.299240,0.0,0.0,0.0,0.178149,0.0,0.0,0.868279,0.0,0.127750,...,1.247164,0.083585,0.000000,0.0,0.0,0.235468,0.455505,0.0,0.620508,0.0
2,0.075862,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,1.718210,0.131872,0.361828,0.0,0.0,0.338238,0.000000,0.0,0.582120,0.0
3,0.189100,0.0,0.0,0.0,0.103399,0.0,0.0,0.000000,0.0,0.000000,...,1.775329,0.000000,0.547958,0.0,0.0,0.000000,0.000000,0.0,0.747348,0.0
4,0.119976,0.0,0.0,0.0,0.141364,0.0,0.0,0.000000,0.0,0.000000,...,1.802694,0.016421,0.451595,0.0,0.0,0.316509,0.000000,0.0,0.705463,0.0


In [0]:
all_id[0]

'images final\\000034954663461785d6392a48f0c002'

In [0]:
for i in range(len(all_id)):
    all_id[i] = all_id[i][13:]

In [0]:
all_id[0]

'000034954663461785d6392a48f0c002'

In [0]:
bottleneck_dataframe['Unique Id'] = all_id
bottleneck_dataframe.head(3)

,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,feature 9,feature 10,...,feature 25080,feature 25081,feature 25082,feature 25083,feature 25084,feature 25085,feature 25086,feature 25087,feature 25088,Unique Id
0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.059623,...,0.749271,0.359782,0.0,0.0,0.452725,0.096178,0.0,0.750572,0.0,000034954663461785d6392a48f0c002
1,2.299240,0.0,0.0,0.0,0.178149,0.0,0.0,0.868279,0.0,0.127750,...,0.083585,0.000000,0.0,0.0,0.235468,0.455505,0.0,0.620508,0.0,00015ad1a28c41e3828a429cf1ca8419
2,0.075862,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.131872,0.361828,0.0,0.0,0.338238,0.000000,0.0,0.582120,0.0,0001799052904228a86842f30a41031f


In [0]:
project_data_sorted = pd.merge(project_data_sorted,bottleneck_dataframe,on="Unique Id",how="left")
project_data_sorted.head(3)

,Unique Id,Image,Brand,Price,Rating,Text,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,feature 9,feature 10,feature 11,feature 12,feature 13,feature 14,feature 15,feature 16,feature 17,feature 18,feature 19,feature 20,feature 21,feature 22,feature 23,feature 24,feature 25,feature 26,feature 27,feature 28,feature 29,feature 30,feature 31,feature 32,feature 33,feature 34,...,feature 25049,feature 25050,feature 25051,feature 25052,feature 25053,feature 25054,feature 25055,feature 25056,feature 25057,feature 25058,feature 25059,feature 25060,feature 25061,feature 25062,feature 25063,feature 25064,feature 25065,feature 25066,feature 25067,feature 25068,feature 25069,feature 25070,feature 25071,feature 25072,feature 25073,feature 25074,feature 25075,feature 25076,feature 25077,feature 25078,feature 25079,feature 25080,feature 25081,feature 25082,feature 25083,feature 25084,feature 25085,feature 25086,feature 25087,feature 25088
0,0697f1714bf848c4b1cacc39a46a4e8b,https://images-na.ssl-images-amazon.com/images...,RoadPro,12.99,4.7,RoadPro RP-100C 10' AM/FM Antenna Coaxial Cabl...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.264963,0.0,0.380408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.097613,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.618720,0.0,0.0,0.0,0.0,0.0,0.102249,0.0,0.099234,0.167977,0.0,0.0,0.338575,0.0,0.072412,0.0,0.0,0.641584,0.013793,0.000000,0.581458,0.194939,0.0,0.0,0.0,0.779111,0.654959,0.0,1.682597,0.150270,0.345941,0.0,0.000000,0.319847,0.0,0.0,0.561830,0.0
1,f04481fec10c4a538634d01e04bb45ed,https://images-na.ssl-images-amazon.com/images...,YSense,9.99,5.0,5-6 Packs Baby Girl Cute Headband - Turban Hea...,0.177580,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.157537,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.323497,0.0,0.0,0.0,0.0,0.0,0.052359,0.0,...,0.0,0.0,0.245931,0.0,0.0,0.0,0.0,0.0,0.112470,0.0,0.248572,0.147189,0.0,0.0,0.574103,0.0,0.000000,0.0,0.0,0.333489,0.000000,0.031226,1.144039,0.188567,0.0,0.0,0.0,0.020072,1.300681,0.0,0.907691,0.000000,0.000000,0.0,0.567571,0.176283,0.0,0.0,0.474543,0.0
2,754577bc25b24641ab1b0aef5e19db73,https://images-na.ssl-images-amazon.com/images...,Wooden Camera,379.18,3.3,Wooden Camera - PL Mount (Weapon/Scarlet-W/Epi...,0.093248,0.0,0.0,0.0,0.191287,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.207076,0.0,0.453692,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.285975,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.570108,0.0,0.0,0.0,0.0,0.0,0.082728,0.0,0.089537,0.202864,0.0,0.0,0.286455,0.0,0.060203,0.0,0.0,0.656395,0.054988,0.000000,0.614037,0.194539,0.0,0.0,0.0,0.950780,0.612823,0.0,1.800421,0.161925,0.283847,0.0,0.000000,0.206060,0.0,0.0,0.629352,0.0


## 2.1 Splitting the data

In [0]:
project_data_sorted_train = project_data_sorted[project_data_sorted['Price'] != 'not_available']
project_data_sorted_test = project_data_sorted[project_data_sorted['Price'] == 'not_available']
print(project_data_sorted_train.shape)
print(project_data_sorted_test.shape)

(44218, 25094)
(2936, 25094)


In [0]:
project_data_sorted_cv = project_data_sorted_train[:int(0.10*len(project_data_sorted_train))]
project_data_sorted_train = project_data_sorted_train[int(0.10*len(project_data_sorted_train)):]
print(project_data_sorted_train.shape)
print(project_data_sorted_cv.shape)

(39797, 25094)
(4421, 25094)


## 2.2 Applying tfidf on text data

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=5)
text_tfidf_train = vectorizer.fit_transform(project_data_sorted_train['Text'])
text_tfidf_cv = vectorizer.transform(project_data_sorted_cv['Text'])
text_tfidf_test = vectorizer.transform(project_data_sorted_test['Text'])
print("Shape of matrix after one hot encodig ",text_tfidf_train.shape)

Shape of matrix after one hot encodig  (39797, 19981)


## 2.3 Brand

In [0]:
project_data_sorted_train['Brand'].nunique()

20995

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
# we use count vectorizer to convert the values into one 
vectorizer = CountVectorizer(vocabulary=project_data_sorted_train['Brand'].unique(), lowercase=False, binary=True)
brand_one_hot_train = vectorizer.fit_transform(project_data_sorted_train['Brand'].values)
brand_one_hot_cv = vectorizer.transform(project_data_sorted_cv['Brand'].values)
brand_one_hot_test = vectorizer.transform(project_data_sorted_test['Brand'].values)

print("Shape of matrix after one hot encodig ",brand_one_hot_train.shape)

Shape of matrix after one hot encodig  (39797, 20995)


## 2.4 Merging the data

In [0]:
print(list(project_data_sorted_train.columns))

['Unique Id', 'Image', 'Brand', 'Price', 'Rating', 'Text', 'feature 1', 'feature 2', 'feature 3', 'feature 4', 'feature 5', 'feature 6', 'feature 7', 'feature 8', 'feature 9', 'feature 10', 'feature 11', 'feature 12', 'feature 13', 'feature 14', 'feature 15', 'feature 16', 'feature 17', 'feature 18', 'feature 19', 'feature 20', 'feature 21', 'feature 22', 'feature 23', 'feature 24', 'feature 25', 'feature 26', 'feature 27', 'feature 28', 'feature 29', 'feature 30', 'feature 31', 'feature 32', 'feature 33', 'feature 34', 'feature 35', 'feature 36', 'feature 37', 'feature 38', 'feature 39', 'feature 40', 'feature 41', 'feature 42', 'feature 43', 'feature 44', 'feature 45', 'feature 46', 'feature 47', 'feature 48', 'feature 49', 'feature 50', 'feature 51', 'feature 52', 'feature 53', 'feature 54', 'feature 55', 'feature 56', 'feature 57', 'feature 58', 'feature 59', 'feature 60', 'feature 61', 'feature 62', 'feature 63', 'feature 64', 'feature 65', 'feature 66', 'feature 67', 'feature 68'

In [0]:
from scipy.sparse import hstack

X_tfidf_train=hstack((text_tfidf_train, brand_one_hot_train, project_data_sorted_train[project_data_sorted_train.columns[6:]].values))
X_tfidf_cv=hstack((text_tfidf_cv, brand_one_hot_cv, project_data_sorted_cv[project_data_sorted_cv.columns[6:]].values))
X_tfidf_test=hstack((text_tfidf_test, brand_one_hot_test, project_data_sorted_test[project_data_sorted_test.columns[6:]].values))

Y_train = project_data_sorted_train['Price'].values
Y_cv = project_data_sorted_cv['Price'].values
Y_test = project_data_sorted_test['Price'].values

In [0]:
print(X_tfidf_train.shape)
print(X_tfidf_cv.shape)
print(X_tfidf_test.shape)

(39797, 66064)
(4421, 66064)
(2936, 66064)


In [0]:
#"Accessing elements in coo_matrix" https://stackoverflow.com/questions/30163830/accessing-elements-in-coo-matrix
X1_tfidf_train=X_tfidf_train.tocsr()
X1_tfidf_val=X_tfidf_cv.tocsr()
X1_tfidf_test=X_tfidf_test.tocsr()

print(X1_tfidf_train.shape)
print(X1_tfidf_val.shape)
print(X1_tfidf_test.shape)

(39797, 66064)
(4421, 66064)
(2936, 66064)


In [0]:
sparse.save_npz(data_path+'X1_tfidf_train.npz', X1_tfidf_train)
sparse.save_npz(data_path+'X1_tfidf_cv.npz', X1_tfidf_val)
sparse.save_npz(data_path+'X1_tfidf_test.npz', X1_tfidf_test)

In [0]:
from scipy import sparse
X1_tfidf_train = sparse.load_npz(data_path + 'X1_tfidf_train.npz')
X1_tfidf_cv = sparse.load_npz(data_path + 'X1_tfidf_cv.npz')
X1_tfidf_test = sparse.load_npz(data_path + 'X1_tfidf_test.npz')

# 3 Modelling

In [0]:
from sklearn.feature_selection import SelectKBest, chi2
selection = SelectKBest(chi2, k=10000).fit(X1_tfidf_train, Y_train)

X_new_train=selection.transform(X1_tfidf_train)
X_new_val=selection.transform(X1_tfidf_val)
X_new_test=selection.transform(X1_tfidf_test)

print(X_new_train.shape)

(39797, 10000)


In [0]:
sparse.save_npz(data_path+'X_new_train.npz', X_new_train)
sparse.save_npz(data_path+'X_new_val.npz', X_new_val)
sparse.save_npz(data_path+'X_new_test.npz', X_new_test)

In [0]:
X_new_train = sparse.load_npz(data_path + 'X_new_train.npz')
X_new_val = sparse.load_npz(data_path + 'X_new_val.npz')
X_new_test = sparse.load_npz(data_path + 'X_new_test.npz')

In [0]:
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from datetime import datetime

params = {
        'n_estimators': [5, 10, 50, 100, 200, 500, 1000],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'learning_rate': [0.01,0.1,0.3,0.5,1],
        'max_depth': [2, 5, 7, 9, 10]
}

In [0]:
#https://www.kaggle.com/tilii7/hyperparameter-grid-search-with-xgboost
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [0]:
xgb = XGBRegressor()
clf = RandomizedSearchCV(xgb, params, cv=5, scoring='neg_mean_squared_error')
start_time = timer(None) # timing starts from this point for "start_time" variable
clf.fit(X_new_train, Y_train)
timer(start_time) # timing ends here for "start_time" variable

[01:16:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:17:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:18:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:19:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:21:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:22:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:23:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:24:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [0]:
clf.best_params_

{'colsample_bytree': 1.0,
 'gamma': 2,
 'learning_rate': 0.1,
 'max_depth': 2,
 'n_estimators': 50}

In [0]:
from sklearn.metrics import r2_score
xgb = XGBRegressor(learning_rate=0.1, max_depth=2, n_estimators=50, colsample_bytree=1.0, gamma=2)
xgb.fit(X1_tfidf_train,Y_train)

[03:45:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1.0, gamma=2,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=2, min_child_weight=1, missing=None, n_estimators=50,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [0]:
print(r2_score(Y_train,xgb.predict(X1_tfidf_train)))

0.61815575556


In [0]:
print(r2_square(Y_cv,xgb.predict(X1_tfidf_val)))

0.54115131787


**Comparison with base model**

In [0]:
Y_base_pred = [sum(list(map(float,Y_train)))/Y_train.shape[0]]*Y_train.shape[0]
len(Y_base_pred)

39797

In [0]:
from sklearn.metrics import r2_score
print(r2_score(Y_train,Y_base_pred))

-2.220446049250313e-16


In [0]:
Y_test_pred = xgb.predict(X1_tfidf_test)